Requirements: Pytorch, mat73, numpy

```pip install mat73```

相關論文： https://ieeexplore.ieee.org/document/9124646

In [ ]:
from os.path import *
import numpy as np
import random
import torch
import torch.nn as nn
import time
import sys
import mat73
import matplotlib.pyplot as plt
from unet import UNet1d

In [ ]:
from scipy.signal import butter, sosfilt, sosfreqz

def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        sos = butter(order, [low, high], analog=False, btype='band', output='sos')
        return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        sos = butter_bandpass(lowcut, highcut, fs, order=order)
        y = sosfilt(sos, data)
        return y

In [ ]:
f = r'../dataset/noscan/170320_CLY/analysis/EyeClose1_noscan.mat'
def norm_ecg(ecg):
    min1, max1 = np.percentile(ecg, [1, 99])
    ecg[ecg>max1] = max1
    ecg[ecg<min1] = min1
    ecg = (ecg - min1)/(max1-min1)
    return ecg


data = mat73.loadmat(f)
eeg_filtered = data['EEG_before_bcg'] * 0
t = time.time()
for ii in range(31):
    eeg_filtered[ii, ...] = butter_bandpass_filter(data['EEG_before_bcg'][ii,:], 1, 40, 5000)

torch.cuda.empty_cache()
device = ('cuda' if torch.cuda.is_available() else 'cpu')
NET = UNet1d(n_channels=1, n_classes=31, nfilter=8).to(device)
#NET = torch.load('pretrainbcg_f8.pt')
#NET.outc = OutConv(8, 31)
NET = NET.to(device)
optimizer = torch.optim.Adam(NET.parameters(), lr=5e-4)
optimizer.zero_grad()
maxlen = data['ECG'].size

loss_list = []
count = 0
ecg = norm_ecg(data['ECG'])
for ii in range(5000):
    if ii % 10 == 0:
        sys.stdout.write('.')
    index = random.randrange(maxlen-20000)
    ECG = ecg[index:(index+20000)]
    EEG = eeg_filtered[:, index:(index+20000)]
    ECG_d = torch.from_numpy(ECG[None, ...][None, ...]).to(device).float()
    EEG_d = torch.from_numpy(EEG[None, ...]).to(device).float()

    # step 3: forward path of UNET
    logits = NET(ECG_d)
    loss = nn.MSELoss()(logits, EEG_d)
    loss_list.append(loss.item())


    # Step 5: Perform back-propagation
    loss.backward() #accumulate the gradients
    optimizer.step() #Update network weights according to the optimizer
    optimizer.zero_grad() #empty the gradients


    if (ii + 1) % 500 == 0: #plot results per 500 iterations
        print('mse loss: ', np.mean(loss_list))
        loss_list = []
        EEG = eeg_filtered[:, :50000]
        ECG = data['ECG'][:50000]
        ECG_d = torch.from_numpy(ECG[None, ...][None, ...]).to(device).float()
        EEG_d = torch.from_numpy(EEG[None, ...]).to(device).float()
        logits = NET(ECG_d)        
        EEG_pred = logits.cpu().detach().numpy()
        plt.figure(figsize=(12, 6), dpi=300)

        plt.plot(EEG[0, ...], 'g')
        plt.plot(EEG[0, ...] - EEG_pred[0, 0, ...], 'r')
        time1 = round(time.time() - t, 1)
        plt.title(f' {time1} seconds')
        plt.show()


# remove BCG from the whole dataset
EEG = eeg_filtered
ECG = data['ECG']
ECG_d = torch.from_numpy(ECG[None, ...][None, ...]).to(device).float()
EEG_d = torch.from_numpy(EEG[None, ...]).to(device).float()
logits = NET(ECG_d)
BCG_pred = logits.cpu().detach().numpy()[0, ...]
EEG_removeBCG_unet = eeg_filtered - BCG_pred

